In [1]:
# Import required libraries
import xarray as xr
from pathlib import Path
import numpy as np
import datetime
import pandas as pd
import cfgrib
import os
import shutil
from datetime import datetime
import matplotlib.pyplot as plt
from scipy.interpolate import griddata

In [2]:
# Read the original datacube
ds_original = xr.open_dataset(Path.home() / 'hdd1/skondylatos/uc3/greece_big_zipped.nc')

In [3]:
ds_original

<xarray.Dataset>
Dimensions:                  (time: 4560, x: 1253, y: 983)
Coordinates:
  * time                     (time) datetime64[ns] 2009-03-06T10:00:00 ... 20...
  * x                        (x) float64 18.7 18.71 18.72 ... 28.88 28.89 28.9
  * y                        (y) float64 42.3 42.29 42.28 ... 34.32 34.31 34.3
    band                     int64 ...
    spatial_ref              int64 ...
Data variables: (12/90)
    1 km 16 days NDVI        (time, y, x) float32 ...
    1 km 16 days EVI         (time, y, x) float32 ...
    ET_500m                  (time, y, x) float32 ...
    LST_Day_1km              (time, y, x) float32 ...
    LST_Night_1km            (time, y, x) float32 ...
    Fpar_500m                (time, y, x) float32 ...
    ...                       ...
    population_density_2016  (y, x) float32 ...
    population_density_2017  (y, x) float32 ...
    population_density_2018  (y, x) float32 ...
    population_density_2019  (y, x) float32 ...
    population_density_2020  (y, x) float32 ...
    population_density_2021  (y, x) float32 ...
Attributes: (12/37)
    CDI:                        Climate Data Interface version 1.9.9rc1 (http...
    Conventions:                CF-1.6
    history:                    Mon Feb 07 16:26:09 2022: cdo mergetime greec...
    Source_Software:            dbinterface.py, dbexport.py, netcdf_handling.py
    creator_name:               Carolina Arias Munoz
    _CoordSysBuilder:           ucar.nc2.dataset.conv.CF1Convention
    ...                         ...
    License:                    Copernicus
    version:                    3.1
    NCO:                        netCDF Operators version 4.7.8 (Homepage = ht...
    cdo_openmp_thread_number:   8
    CDO:                        Climate Data Operators version 1.9.9rc1 (http...
    crs:                        EPSG:4326

In [148]:
def create_new_ds(ds1, ds2):
    ds = xr.merge([ds1, ds2], compat = 'override')
    x = ds.isel(step = slice(0,24))
    x = x.interp(longitude = ds_original.x.values, latitude = ds_original.y.values, method = 'nearest')
    x['wind_speed'] = np.sqrt(x['u10'] ** 2 + x['v10'] ** 2)
    x['d_2m'] = (243.04 * ( np.log(x['r2']/100) + ( ( 17.625*(x['t2m'] -273.15) ) / (243.04+x['t2m']-273.15) ) ) / (17.625 - np.log(x['r2']/100) - ( (17.625* (x['t2m']-273.15) ) / (243.04+x['t2m']-273.15)))) + 273.15
    max_ds = xr.Dataset(
        data_vars=dict(
            max_t2m=(["y", "x"], x['t2m'].max(dim = 'step').data),
            min_relhum_2m=(["y", "x"], x['r2'].min(dim = 'step').data),
            max_d2m=(["y", "x"], x['d_2m'].max(dim = 'step').data),
            max_wind_speed=(["y", "x"], x['wind_speed'].max(dim = 'step').data)
        ),
        coords=dict(
            x=ds_original['x'].values,
            y=ds_original['y'].values
        )
    )
    time2 = x.valid_time[0].values
    max_ds = max_ds.expand_dims(time = [time2])
    max_ds['min_relhum_2m'] = max_ds['min_relhum_2m'] / 100
    return max_ds

In [ ]:
months_31days = [5, 7, 8, 10]
daily = []
for month in range(5,11):
    for day in range(1,32):
        if(month < 10):
            if month in months_31days:
                if day < 10:
                    path = 'jh-shared/METEO_DATA/PAPOU_DATA/MOLOCH_DATA/moloch_210' + str(month) + '0' + str(day) + '00' + '.grib'
                    path = Path.home() / path
                    tmp1 = xr.open_dataset(path,engine = 'cfgrib', indexpath = '',backend_kwargs={'filter_by_keys': { 'stepUnits': 1,'typeOfLevel' :  'heightAboveGround', 'level' : 2}})
                    tmp2 = xr.open_dataset(path,engine = 'cfgrib', indexpath = '',backend_kwargs={'filter_by_keys': { 'stepUnits': 1,'typeOfLevel' :  'heightAboveGround', 'level' : 10}})
                    daily.append(create_new_ds(tmp1, tmp2))
                else:
                    if month == 8 and day == 22:
                        continue
                    if month == 8 and day == 23:
                        continue
                    path = 'jh-shared/METEO_DATA/PAPOU_DATA/MOLOCH_DATA/moloch_210' + str(month) + str(day) + '00' + '.grib'
                    path = Path.home() / path
                    tmp1 = xr.open_dataset(path,engine = 'cfgrib', indexpath = '',backend_kwargs={'filter_by_keys': { 'stepUnits': 1,'typeOfLevel' :  'heightAboveGround', 'level' : 2}})
                    tmp2 = xr.open_dataset(path,engine = 'cfgrib', indexpath = '',backend_kwargs={'filter_by_keys': { 'stepUnits': 1,'typeOfLevel' :  'heightAboveGround', 'level' : 10}})
                    daily.append(create_new_ds(tmp1, tmp2))
            else:
                if day < 10:
                    path = 'jh-shared/METEO_DATA/PAPOU_DATA/MOLOCH_DATA/moloch_210' + str(month) + '0' + str(day) + '00' + '.grib'
                    path = Path.home() / path
                    tmp1 = xr.open_dataset(path,engine = 'cfgrib', indexpath = '',backend_kwargs={'filter_by_keys': { 'stepUnits': 1,'typeOfLevel' :  'heightAboveGround', 'level' : 2}})
                    tmp2 = xr.open_dataset(path,engine = 'cfgrib', indexpath = '',backend_kwargs={'filter_by_keys': { 'stepUnits': 1,'typeOfLevel' :  'heightAboveGround', 'level' : 10}})
                    daily.append(create_new_ds(tmp1, tmp2))
                elif day < 31:
                    path = 'jh-shared/METEO_DATA/PAPOU_DATA/MOLOCH_DATA/moloch_210' + str(month) + str(day) + '00' + '.grib'
                    path = Path.home() / path
                    tmp1 = xr.open_dataset(path,engine = 'cfgrib', indexpath = '',backend_kwargs={'filter_by_keys': { 'stepUnits': 1,'typeOfLevel' :  'heightAboveGround', 'level' : 2}})
                    tmp2 = xr.open_dataset(path,engine = 'cfgrib', indexpath = '',backend_kwargs={'filter_by_keys': { 'stepUnits': 1,'typeOfLevel' :  'heightAboveGround', 'level' : 10}})
                    if month == 9 and day == 18:
                        tmp1 = tmp1.isel(time=1)
                        tmp2 = tmp2.isel(time=1)
                    daily.append(create_new_ds(tmp1, tmp2))
        else:
            if month in months_31days:
                if day < 10:
                    path = 'jh-shared/METEO_DATA/PAPOU_DATA/MOLOCH_DATA/moloch_21' + str(month) + '0' + str(day) + '00' + '.grib'
                    path = Path.home() / path
                    tmp1 = xr.open_dataset(path,engine = 'cfgrib', indexpath = '',backend_kwargs={'filter_by_keys': { 'stepUnits': 1,'typeOfLevel' :  'heightAboveGround', 'level' : 2}})
                    tmp2 = xr.open_dataset(path,engine = 'cfgrib', indexpath = '',backend_kwargs={'filter_by_keys': { 'stepUnits': 1,'typeOfLevel' :  'heightAboveGround', 'level' : 10}})
                    daily.append(create_new_ds(tmp1, tmp2))
                else:
                    path = 'jh-shared/METEO_DATA/PAPOU_DATA/MOLOCH_DATA/moloch_21' + str(month) + str(day) + '00' + '.grib'
                    path = Path.home() / path
                    tmp1 = xr.open_dataset(path,engine = 'cfgrib', indexpath = '',backend_kwargs={'filter_by_keys': { 'stepUnits': 1,'typeOfLevel' :  'heightAboveGround', 'level' : 2}})
                    tmp2 = xr.open_dataset(path,engine = 'cfgrib', indexpath = '',backend_kwargs={'filter_by_keys': { 'stepUnits': 1,'typeOfLevel' :  'heightAboveGround', 'level' : 10}})
                    daily.append(create_new_ds(tmp1, tmp2))
            else:
                if day < 10:
                    path = 'jh-shared/METEO_DATA/PAPOU_DATA/MOLOCH_DATA/moloch_21' + str(month) + '0' + str(day) + '00' + '.grib'
                    path = Path.home() / path
                    tmp1 = xr.open_dataset(path,engine = 'cfgrib', indexpath = '',backend_kwargs={'filter_by_keys': { 'stepUnits': 1,'typeOfLevel' :  'heightAboveGround', 'level' : 2}})
                    tmp2 = xr.open_dataset(path,engine = 'cfgrib', indexpath = '',backend_kwargs={'filter_by_keys': { 'stepUnits': 1,'typeOfLevel' :  'heightAboveGround', 'level' : 10}})
                    daily.append(create_new_ds(tmp1, tmp2))
                elif day < 31:
                    path = 'jh-shared/METEO_DATA/PAPOU_DATA/MOLOCH_DATA/moloch_21' + str(month) +  str(day) + '00' + '.grib'
                    path = Path.home() / path
                    tmp1 = xr.open_dataset(path,engine = 'cfgrib', indexpath = '',backend_kwargs={'filter_by_keys': { 'stepUnits': 1,'typeOfLevel' :  'heightAboveGround', 'level' : 2}})
                    tmp2 = xr.open_dataset(path,engine = 'cfgrib', indexpath = '',backend_kwargs={'filter_by_keys': { 'stepUnits': 1,'typeOfLevel' :  'heightAboveGround', 'level' : 10}})
                    daily.append(create_new_ds(tmp1, tmp2))

In [186]:
x = xr.merge([daily[0], daily[1]])
for i in range(2,len(daily)):
    x = xr.merge([x, daily[i]])

In [187]:
x

<xarray.Dataset>
Dimensions:         (time: 182, y: 983, x: 1253)
Coordinates:
  * time            (time) datetime64[ns] 2021-05-01 2021-05-02 ... 2021-10-31
  * x               (x) float64 18.7 18.71 18.72 18.73 ... 28.88 28.89 28.9
  * y               (y) float64 42.3 42.29 42.28 42.27 ... 34.32 34.31 34.3
Data variables:
    max_t2m         (time, y, x) float32 293.9 293.8 293.8 ... 294.4 294.4 294.4
    min_relhum_2m   (time, y, x) float32 0.5192 0.5233 0.5233 ... 0.7217 0.7217
    max_d2m         (time, y, x) float32 288.4 288.4 288.4 ... 292.0 292.0 292.0
    max_wind_speed  (time, y, x) float32 8.199 7.81 7.81 ... 9.779 9.834 9.834

In [188]:
path = Path.home() / 'hdd1/diogenis/observatory/moloch_meteo.nc'
x.to_netcdf(path)